# CS5242

## Load Data

In [16]:
from data import Data

train_data = Data(data_type='train', max_files=1000)
test_data = Data(data_type='test')

print("Train data: x:"+ str(train_data.x.shape)+ ' y:'+str(train_data.y.shape))
print("Test data: x:"+ str(test_data.x.shape)+ ' y:'+str(test_data.y.shape))



Loading train data 1000/1000: 100%|███████████████████████████████████████████████████████████████ [ time left: 00:00 ]
Loading test data 6051/6051: 100%|████████████████████████████████████████████████████████████████ [ time left: 00:00 ]


Train data: x:(1000, 1000, 102) y:(1000, 1)
Test data: x:(6051, 1000, 102) y:(6051, 1)


## Model

In [17]:
import tensorflow as tf

model = tf.keras.Sequential([
    # tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection multiple                  85504     
_________________________________________________________________
dense_2 (Dense)              multiple                  8256      
_________________________________________________________________
dense_3 (Dense)              multiple                  65        
Total params: 93,825
Trainable params: 93,825
Non-trainable params: 0
_________________________________________________________________


## Save Checkpoint

In [18]:
import os
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [19]:
history = model.fit(x=train_data.x, y=train_data.y, validation_split=0.1, epochs=5, callbacks=[checkpoint_callback])


Train on 900 samples, validate on 100 samples
Epoch 1/5
900/900 [==============================] - ETA: 3:29 - loss: 0.7555 - accuracy: 0.34 - ETA: 2:35 - loss: 0.7362 - accuracy: 0.39 - ETA: 2:11 - loss: 0.7146 - accuracy: 0.43 - ETA: 1:59 - loss: 0.7002 - accuracy: 0.45 - ETA: 1:52 - loss: 0.6961 - accuracy: 0.46 - ETA: 1:42 - loss: 0.6983 - accuracy: 0.45 - ETA: 1:35 - loss: 0.6971 - accuracy: 0.45 - ETA: 1:29 - loss: 0.6931 - accuracy: 0.47 - ETA: 1:23 - loss: 0.6935 - accuracy: 0.47 - ETA: 1:17 - loss: 0.6917 - accuracy: 0.48 - ETA: 1:11 - loss: 0.6828 - accuracy: 0.51 - ETA: 1:06 - loss: 0.6811 - accuracy: 0.51 - ETA: 1:01 - loss: 0.6806 - accuracy: 0.52 - ETA: 56s - loss: 0.6802 - accuracy: 0.5268 - ETA: 52s - loss: 0.6772 - accuracy: 0.541 - ETA: 48s - loss: 0.6769 - accuracy: 0.550 - ETA: 43s - loss: 0.6747 - accuracy: 0.549 - ETA: 39s - loss: 0.6746 - accuracy: 0.557 - ETA: 35s - loss: 0.6732 - accuracy: 0.562 - ETA: 31s - loss: 0.6707 - accuracy: 0.568 - ETA: 27s - loss: 0.6

## Restore Weights

In [12]:
checkpoint_dir = './training_checkpoints'
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [20]:
test_predict = model.predict(test_data.x)

In [21]:
print(test_predict.shape)

(6051, 1)


## Save Prediction to CSV

In [23]:
import csv

output_file = 'result.csv'
with open(output_file, 'wt', newline='', encoding='utf-8') as output_file:
    csv_writer = csv.writer(output_file)
    csv_writer.writerow(('Id', 'Predicted'))
    for id, predict in enumerate(test_predict):
        csv_writer.writerow((id, predict[0]))